# Preprocessing data for the Data Challenge

In [1]:
!pip install langdetect --quiet

In [2]:
!pip install googletrans --quiet

In [3]:
!pip install google_trans_new --quiet

In [4]:
!pip install deep_translator --quiet

In [5]:
import pandas as pd
import numpy as np
import nltk


import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

import time
import re
from langdetect import detect
from googletrans import Translator
from google_trans_new import google_translator 

from collections import Counter

import gc

from deep_translator import (GoogleTranslator)


Bad key text.latex.preview in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 123 ('text.latex.preview : False')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key mathtext.fallback_to_cm in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 155 ('mathtext.fallback_to_cm : True  # When True, use symbols from the Computer Modern')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.5.1/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.jpeg_quality in file /opt/anaconda3/lib/python3.9/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle, line 418 ('savefig.jpeg_quality: 95       # when a jpeg is saved, the default quality parameter.')
You proba

In [6]:
FILENAME = "../data/train_series.csv"
FILENAME_ECB = "../data/ecb_data.csv"
FILENAME_FED = "../data/fed_data.csv"

In [7]:
ecb = pd.read_csv(FILENAME_ECB, index_col=0)
fed = pd.read_csv(FILENAME_FED, index_col=0)

In [8]:
ecb.head(1)

,title,speaker,text
0,Comments by Yves Mersch at Financial Services ...,Yves Mersch,Comments by Yves Mersch at Financial Service...


In [9]:
fed.head(1)

,title,speaker,text
0,The Importance of Economic Education and Finan...,Governor Frederic S. Mishkin,As ...


## Apply the pipeline for english text

In [10]:
# Before dealing with N/A because otherwise, this edit would just undo the other.
# We can use this to remove the "TRANSCRIPT" and "SPEECH" tags.
# We also notice that most of the time, the content is preceded by a date. It would be useful to make a date parser to remove the first chunk.

def numbered_reference_removal(text):
    # refs are typically in the form [n] in the text.
    matches = re.findall(r'\[[0-9]+?\]', text)
    counter = Counter(matches)
    for match in matches:
        if counter[match] != 2:
            # print(f"Didn't find a reference twice in the text, but {counter[match]} times. Cannot remove.")
            # print(text[:2000], '\n ... \n',text[-2000:])
            # print()
            return text
    
    assert len(matches)%2 == 0
    if len(matches) == 0:
        return text
    N = len(matches)//2
    is_ordered = True
    n = 0
    for s in matches:
        is_ordered = (s == f'[{n%N+1}]')
        if not is_ordered : 
            # print(f"Not ordered, {str(matches)}")
            return text
        n += 1
    
    # Remove all references after [1]
    res = re.findall(
        r'^.*?\[1\].+?\[1\]',
        text
    )
    return res[0]

def reference_removal_en(text):
    res = re.sub(r'(?i)see also.*', '', text)
    res = re.sub(r'References.*', '', res)
    res = re.sub("\[[0-9]+\]", "", res)
    return res

def first_date_extractor(text):
    if len(text) > 0:
        res = re.sub('^(.*?)[1-9][0-9]* (?:Jan(?:uary)?|Feb(?:ruary)?|Mar(?:ch)?|'
                     + 'Apr(?:il)?|May|Jun(?:e)?|Jul(?:y)?|Aug(?:ust)?|Sep(?:tember)?|Oct(?:ober)?|(Nov|Dec)(?:ember)?) (19|20)[0-9][0-9]',
                     '', text)
        return res

def remove_title(x):
    if not x["text"] is None and isinstance(x["text"], str):
        res = re.sub(x["title"], '', x["text"]).strip()
        return res
    else:
        return x["title"]

def website_remover(text):
    # Remove websites
    regex = "((http|https)://)(www.)?" \
        + "[a-zA-Z0-9@:%._\\+~#?&//=]{2,256}\\.[a-z]" \
        + "{2,6}\\b([-a-zA-Z0-9@:%._\\+~#?&//=]*)"
    res = re.sub(regex, "", text).strip()
    return res

def tag_removal(text):
    # Remove tags
    res = re.sub('SPEECH', '', text)
    res = re.sub('TRANSCRIPT', '', res)
    res = re.sub("Introduction", "", res)
    res = re.sub("Summary", "", res)
    return res

def pipeline_en(x):
    res = remove_title(x)
    if res is None:
        return res
    res = numbered_reference_removal(res)
    res = reference_removal_en(res)
    res = tag_removal(res).strip()
    res = first_date_extractor(res).strip()

    # print("================================================")
    # print(res[:200])
    # print("...")
    # print(res[-500:])
    return res

ecb["text_preprocessed"] = ecb.apply(pipeline_en, axis=1)

In [11]:
#ecb["text_preprocessed"][31]

## Detect the text in english, deutsch, french, spanish or italian

In [12]:
ecb["lang"] = ecb["text_preprocessed"].apply(lambda x : detect(x[:500]))
fed["lang"] = fed["text"].apply(lambda x : detect(x[:500]))

In [13]:
ecb["lang"].value_counts()

en    1646
de      75
fr      31
es      16
it       4
Name: lang, dtype: int64

In [14]:
fed["lang"].value_counts()

en    739
Name: lang, dtype: int64

In [15]:
ecb.columns

Index(['title', 'speaker', 'text', 'text_preprocessed', 'lang'], dtype='object')

In [16]:
'''
Calculer la somme de la longueur de la chaîne de caractères de cette colonne pour 
toutes les lignes qui ne sont pas en langue anglaise.
L'argument skipna=True de la méthode sum() indique que les valeurs manquantes (NaN) 
doivent être ignorées dans le calcul de la somme
'''
ecb[ecb["lang"] != "en"]["text_preprocessed"].str.len().sum(skipna=True)

2565448

In [17]:
num_words_total = ecb["text_preprocessed"].str.split().apply(len).sum(skipna=True)
print("Au total, le premier texte contient {} mots.".format(num_words_total))

Au total, le premier texte contient 4610038 mots.


In [18]:
first_text = ecb[ecb["lang"] != "en"].iloc[0]["text_preprocessed"]
num_words_first_line = len(first_text.split())
print("Le premier texte contient {} mots.".format(num_words_first_line))

Le premier texte contient 2573 mots.


## Split all texts in a list of texts of 4500 characters maximum

In [19]:
italian_text = ecb[ecb["lang"] == "it"]["text_preprocessed"]
english_text = ecb[ecb["lang"] == "en"]["text_preprocessed"]
spanish_text = ecb[ecb["lang"] == "es"]["text_preprocessed"]
french_text = ecb[ecb["lang"] == "fr"]["text_preprocessed"]
deutsch_text = ecb[ecb["lang"] == "de"]["text_preprocessed"]
deutsch_text.index

Int64Index([  31,   40,   48,   66,  145,  151,  179,  200,  218,  255,  257,
             261,  275,  287,  302,  348,  363,  399,  402,  422,  442,  453,
             501,  509,  521,  532,  535,  541,  646,  662,  665,  685,  689,
             798,  829,  844,  857,  963,  978, 1001, 1005, 1008, 1012, 1026,
            1070, 1085, 1089, 1094, 1108, 1117, 1127, 1157, 1174, 1193, 1213,
            1234, 1261, 1298, 1310, 1324, 1335, 1395, 1400, 1464, 1467, 1521,
            1562, 1585, 1589, 1609, 1652, 1708, 1727, 1762, 1765],
           dtype='int64')

In [51]:
def split_text(text):
    nltk.download('punkt')
    
    sentences = nltk.sent_tokenize(text)
    new_text = []
    current_text = ''
    for sentence in sentences:
        if len(current_text + sentence) <= 4500:
            current_text += sentence
        else:
            if current_text[-1] != '.':
                last_period = current_text.rfind('.')
                if last_period != -1:
                    new_text.append(current_text[:last_period+1])
                    current_text = current_text[last_period+1:]
            new_text.append(current_text)
            current_text = sentence
    if current_text != '':
        new_text.append(current_text)
    
    
    if len(new_text[-1]) < 200 and len(new_text) > 1:
        new_text[-2] = new_text[-2] + new_text[-1]
        new_text.pop(-1)
    
    return new_text

In [52]:
def df_with_split_text(df):
    nltk.download('punkt')
    df_split_text = df.copy()

    for index in range(df_split_text.shape[0]):
        new_text = split_text(df_split_text.iloc[index]['text_preprocessed'])
        df_split_text.iloc[index]['text_preprocessed'] = new_text

    return df_split_text

In [53]:
ecb_split_text = df_with_split_text(ecb)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/solenedebuysere/nltk_data...
[nltk_data]   Package punkt is already up-t

In [54]:
len(ecb_split_text.iloc[0]['text_preprocessed'])

2

In [55]:
ecb_split_text['text_preprocessed'].isna().sum()

0

## Translate all text in english

In [56]:
from langdetect import detect

In [88]:
def translate_text(text, index):
    translated_text = text
    
    if index is deutsch_text.index : 
        lang_text = 'german'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index is italian_text.index : 
        lang_text = 'it'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index is spanish_text.index : 
        lang_text = 'es'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
        
    if index is french_text.index : 
        lang_text = 'fr'
        translated_text = GoogleTranslator(source=lang_text, target='en').translate(text=text)
    
    if index is english_text.index:
        translated_text = text
    
    return translated_text

In [89]:
def translate_df(df):
    list_translated_text = []
    df_translated = df.copy()

    for index in range(df_translated.shape[0]):
        
        for i in range(len(df_translated.iloc[index]['text_preprocessed'])):
            translated_text_i = translate_text(df_translated.iloc[index]['text_preprocessed'][i], index)
            list_translated_text.append(translated_text_i)
        df_translated.iloc[index]['text_preprocessed'] = list_translated_text

    return df_translated

In [ ]:
fed

In [96]:
df_translated_text = translate_df(ecb_split_text)
#df_translated_text.iloc[31]['text_preprocessed']


In [90]:
ecb_split_text['text_preprocessed'].apply(lambda x: ''.join(x)).str.len().describe()

count     1772.000000
mean     16535.196388
std       9542.230858
min         22.000000
25%      10048.500000
50%      15282.000000
75%      21886.250000
max      77554.000000
Name: text_preprocessed, dtype: float64

## Take the new dataframe translated to make each list (of several elements) as one string

In [97]:
def concat_text(df):
    
    for index in range(df.shape[0]):
        concatenated_text = []
        concatenated_text.append(''.join(df.iloc[index]['text_preprocessed']))
        df.iloc[index]['text_preprocessed'] = concatenated_text
    
    return df

In [2]:
#df_clean = concat_text(df_translated_text)
#df_clean.iloc[31]['text_preprocessed']
